# Preparación de datos
## Importar bibliotecas

In [2]:
import pandas as pd 
import numpy as np 
import scipy as sp
import re
from tqdm import tqdm


from unidecode import unidecode

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder


from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif

## Importación de Datos 

In [3]:
df = pd.read_excel("../datasets/raw/Diccionario de Datos STEAM.xlsx", sheet_name="Hoja 1",  header=None,engine="openpyxl")
print(df.shape)

(29, 3)


In [1]:
# Filtrar las filas donde el campo en la columna 0 contiene '.json'
filtro_title = df[df[0].str.contains('.json', na=False)]
filtro_titles =filtro_title[0].unique()
filtro_titles

NameError: name 'df' is not defined

In [5]:
# Encontrar las filas donde comienzan nuevas listas
mask = df[0] == 'Columna'

# Crear un identificador único para cada grupo de listas
group_id = mask.cumsum()

# Agrupar por el identificador único y concatenar las listas
grouped_df = df.groupby(group_id).agg(lambda x: x.tolist()).reset_index(drop=True)

# Establecer la primera fila como encabezado y eliminar la fila de encabezado original y las filas con 'Columna'
grouped_df.columns = grouped_df.iloc[0]
grouped_df = grouped_df.iloc[1:].reset_index(drop=True)

# Transponer la fila para que las listas se conviertan en columnas
dataframes_list = []

for i in range(len(grouped_df)):
    row_values = grouped_df.iloc[i, :].tolist()
    df_temp = pd.DataFrame(row_values).T
    df_temp.columns = df_temp.iloc[0]
    df_temp = df_temp[1:].reset_index(drop=True)
    df_temp.dropna(inplace=True)
    dataframes_list.append(df_temp)

# Concatenar todos los DataFrames en uno solo
result_df = pd.concat(dataframes_list, axis=0, keys=range(1, len(dataframes_list)+1))

# Asignar los nombres a las filas
user_items = result_df[result_df.index.get_level_values(0) == 1].copy()
user_reviews = result_df[result_df.index.get_level_values(0) == 2].copy()
user_items = result_df[result_df.index.get_level_values(0) == 3].copy()
# Restablecer los índices si es necesario
#nuevo_df.reset_index(drop=True, inplace=True)

0            Columna                        Descripción  \
1 0        publisher  Empresa publicadora del contenido   
  1           genres               Genero del contenido   
  2         app_name               Nombre del contenido   
  3            title               Titulo del contenido   
  4              url   URL de publicación del contenido   
  5     release_date               Fecha de lanzamiento   
  6             tags             etiquetas de contenido   
  7   discount_price                precio de descuento   
  8      reviews_url               Reviews de contenido   
  9            specs                   Especificaciones   
  10           price               Precio del contenido   
  11    early_access                    acceso temprano   
  12              id   identificador unico de contenido   
  13       developer                      Desarrollador   
  14       metascore               Score por metacritic   
2 0          user_id     identificador unico de usuario   
  1         user_url             URL perfil del usuario   
  2          reviews  Review de usuario en formato Json   
3 0          user_id     identificador unico de usuario   
  1         user_url             URL perfil del usuario   
  2            items   Items de usuario en formato Json   

0                                               Ejemplo  
1 0            [Ubisoft,Dovetail Games - Trains,Degica]  
  1   [Action, Adventure, Racing, Simulation, Strategy]  
  2                [Warzone, Soundtrack, Puzzle Blocks]  
  3   [The Dream Machine: Chapter 4 , Fate/EXTELLA -...  
  4   http://store.steampowered.com/app/761140/Lost_...  
  5                                        [2018-01-04]  
  6   [Simulation, Indie, Action, Adventure, Funny, ...  
  7                                 [22.66, 0.49, 0.69]  
  8   http://steamcommunity.com/app/681550/reviews/?...  
  9   [Multi-player, Co-op, Cross-Platform Multiplay...  
  10            [4.99, 9.99, Free to Use, Free to Play]  
  11                                      [False, True]  
  12                           [761140, 643980, 670290]  
  13        [Kotoshiro, Secret Level SRL, Poolians.com]  
  14                                   [80, 74, 77, 75]  
2 0           [76561197970982479, evcentric, maplemage]  
  1              http://steamcommunity.com/id/evcentric  
  2   {'funny': '', posted': 'Posted September 8, 20...  
3 0           [76561197970982479, evcentric, maplemage]  
  1              http://steamcommunity.com/id/evcentric  
  2   {'item_id': '273350', 'item_name': 'Evolve Sta...

In [181]:
import requests
import shutil
import os
import gzip

def download_and_extract(url, destination_folder, extract=False):
    os.makedirs(destination_folder, exist_ok=True)
    response = requests.get(url, stream=True)
    file_name = os.path.join(destination_folder, os.path.basename(url))

    with open(file_name, "wb") as file:
        shutil.copyfileobj(response.raw, file)

    if extract:
        with gzip.open(file_name, 'rb') as f_in:
            with open(file_name[:-3], 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        os.remove(file_name)
        return file_name[:-3]
    else:
        return file_name

def main():
    reviews_url = "http://cseweb.ucsd.edu/~wckang/steam_reviews.json.gz"
    games_url = "http://cseweb.ucsd.edu/~wckang/steam_games.json.gz"
    destination_folder = "../datasets/raw"

    reviews_file = download_and_extract(reviews_url, destination_folder)
    games_file = download_and_extract(games_url, destination_folder)

    print(os.listdir(destination_folder))

if __name__ == "__main__":
    main()


['australian_users_items.json.gz', 'australian_user_reviews.json', 'Diccionario de Datos STEAM.xlsx', 'output_steam_games.json', 'steam_games.json.gz', 'steam_reviews.json.gz']


In [ ]:
indices_unicos = result_df.index.get_level_values(0).unique()
nombres_asignados = {indice: filtro_titles[i % len(filtro_titles)] for i, indice in enumerate(indices_unicos)}
result_df.index = result_df.index.get_level_values(0).map(nombres_asignados)

- Contexto de análisis